In [2]:
# package load
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
from geopy.geocoders import Nominatim
from tqdm import tqdm

# for korean plotting
plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['axes.unicode_minus'] = False

# Pandas option
pd.set_option('display.max_columns', None)

In [4]:
result = pd.read_csv('STKDE/result/DJ_sb_3000_tb_72.csv')
grid = gpd.read_file('data/CENSUS/grid_DJ.gpkg')
result_grid = result.groupby(['X','Y'], as_index=False).sum()
result_grid = gpd.GeoDataFrame(result_grid['density'], geometry=gpd.points_from_xy(result_grid.X, result_grid.Y), crs='epsg:5181')

# 격자에 병합
grid['density'] = 0.0
for i in range(len(grid)):
    grid.iat[i, 2] = result_grid.iat[np.where(result_grid.geometry.intersects(grid.geometry[i]))[0][0],0]

## Merge with other data

In [7]:
df_full = gpd.read_file('df_full.gpkg').set_index('GRID_1K_CD')
df_full = grid.set_index('GRID_1K_CD').join(df_full[df_full.columns.drop('geometry')])
df_full.to_csv('df_for_regression.csv', index=True)